Домашнє завдання

На цьому тижні ми вивчили як працюють рекомендаційні системи. Пропонуємо вам познайомитись з бібліотекою surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.

Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.

In [5]:
import pandas as pd
from surprise import Dataset, Reader, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

# Завантаження даних
data = Dataset.load_builtin('ml-100k', prompt=True)

# Поділ даних на тренувальні та тестові
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Створення та навчання моделі SVD
model_svd = SVD()
model_svd.fit(trainset)
predictions_svd = model_svd.test(testset)
print('SVD RMSE:', accuracy.rmse(predictions_svd))

# Пошук найкращих параметрів для SVD
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.05]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1, pre_dispatch='2*n_jobs')
gs.fit(data)
print('Best SVD params:', gs.best_params['rmse'])

# Крос-валідація SVD++
model_svdpp = SVDpp()
results_svdpp = cross_validate(model_svdpp, data, measures=['rmse'], cv=3, verbose=True)
print('SVD++ RMSE:', pd.DataFrame.from_dict(results_svdpp).mean()['test_rmse'])

# Крос-валідація NMF
model_nmf = NMF()
results_nmf = cross_validate(model_nmf, data, measures=['rmse'], cv=3, verbose=True)
print('NMF RMSE:', pd.DataFrame.from_dict(results_nmf).mean()['test_rmse'])



RMSE: 0.9408
SVD RMSE: 0.9408317867738821
Best SVD params: {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05}
Evaluating RMSE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9270  0.9268  0.9251  0.9263  0.0008  
Fit time          14.18   14.25   14.17   14.20   0.04    
Test time         5.43    5.13    5.54    5.36    0.17    
SVD++ RMSE: 0.9262829476939141
Evaluating RMSE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9666  0.9780  0.9768  0.9738  0.0051  
Fit time          1.69    1.56    1.52    1.59    0.07    
Test time         0.31    0.15    0.28    0.25    0.07    
NMF RMSE: 0.9738146542355164


Отже, якщо потрібна максимальна точність – потрібно вибрати SVD++, якщо важлива швидкість – потрібно вибрати NMF, а якщо потрібен баланс між точністю і швидкістю – вибір SVD.